# Clean up West Virginia Ownership Data

The notebook below combines data from the CMS QCOR database and West Virginia's OHFLAC for the analysis.

In [1]:
import pandas as pd

## Load OHFLAC Data

The public CMS data doesn't contain information about ownership. In order to fix that issue I downloaded the information about all of the facilities from [OHFLAC search site](https://ohflac.wvdhhr.org/Apps/Lookup/FacilitySearch).

In [2]:
INTERMEDIATE_OWNERSHIP_DIR = "../../data/ownership/intermediate/"

In [3]:
wv_icfs = pd.read_csv(INTERMEDIATE_OWNERSHIP_DIR + "wv-icf-info.csv")

In [4]:
wv_icfs.head()

,Facility Name,Legal Name,Administrator,State Key,Operation Status,Opened,Closed,Physical Address,Physical Address 2,City,...,License Expires,Licensed Beds,Certified Beds,SSI Beds,Total Beds,Medicare,Medicaid,SSI,Longitude,Latitude
0,1204 S. KANAWHA GROUP HOME,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",NaN,505312,Active,1987/02/17,NaN,1204 S. KANAWHA STREET,NaN,Beckley,...,NaN,8,8,0,8,No,Yes,No,NaN,NaN
1,5TH AVENUE GROUP HOME,"RSCR WEST VIRGINIA, INCORPORATED",NaN,505375,Active,1981/10/06,NaN,916 FIFTH AVENUE,NaN,Saint Albans,...,NaN,9,9,0,9,No,Yes,No,NaN,NaN
2,6TH STREET WEST GROUP HOME,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",NaN,505263,Active,2006/11/13,NaN,1036 6TH STREET WEST,NaN,Huntington,...,NaN,4,4,0,4,No,Yes,No,NaN,NaN
3,811 S. KANAWHA GROUP HOME,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",NaN,505312,Active,1987/02/17,NaN,811 S. KANAWHA STREET,NaN,Beckley,...,NaN,8,8,0,8,No,Yes,No,NaN,NaN
4,ACCOVILLE GROUP HOME,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",NaN,505313,Active,1990/10/04,NaN,30 LULA SMOOTH DRIVE,NaN,Accoville,...,NaN,8,8,0,8,No,Yes,No,NaN,NaN


In [5]:
len(wv_icfs)

67

## Load QCOR Data

In [6]:
fac_df = pd.read_csv("../../data/qcor/facilities.csv")

In [7]:
wv_facs = fac_df.loc[lambda x: (x["state"] == "WV")].copy()

In [8]:
wv_facs.head()

,name,provider_id,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date
6463,5TH AVENUE GROUP HOME,51G002,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"916 FIFTH AVENUE\nSAINT ALBANS, WV 25177",304 720-0015,1981-10-06,9,No,For Profit,NaN,NaN
6464,POTOMAC CENTER,51G003,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"ONE BLUE STREET\nROMNEY, WV 26757",304 822-3861,1983-11-09,24,No,Non-Profit,NaN,NaN
6465,CROSS LANES GROUP HOME,51G007,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"5202 LINDA VISTA DRIVE\nCROSS LANES, WV 25313",304 776-2005,1986-11-17,8,No,For Profit,NaN,NaN
6466,EIGHTH AVENUE GROUP HOME,51G008,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"1519 8TH AVENUE\nHUNTINGTON, WV 25701",304 523-0177,1987-03-06,8,No,For Profit,NaN,NaN
6467,VIRGINIA AVENUE GROUP HOME,51G009,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"821 VIRGINIA AVENUE\nHUNTINGTON, WV 25701",304 523-0196,1987-03-06,8,No,For Profit,NaN,NaN


In [9]:
len(wv_facs)

69

There are two facilities in the QCOR data not in the West Virgina data. One was closed and the other was opened in 2021. Information for Gaboya Place Group Home does exist on the [OHFLAC website](https://ohflac.wvdhhr.org/Apps/Lookup/FacilityDetails/1062). So we will use that to update the data for it later.

In [10]:
wv_facs.loc[
    lambda x: ~x["name"].isin(wv_icfs["Facility Name"])
]

,name,provider_id,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date
6480,GABOYA PLACE GROUP HOME,51G023,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"201 SOUTH WATER STREET\nMARTINSBURG, WV 25401",304 263-4601,1988-05-19,8,No,For Profit,"Vol-Merg, Close",2019-12-02
6531,HAYNIE ROAD ICF,51G077,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"2854 HAYNIE ROAD\nHUNTINGTON, WV 25704",304 972-0084,2021-07-14,4,No,For Profit,NaN,NaN


In [11]:
wv_facs_with_owner = (
    wv_facs
    .merge(
        wv_icfs[["Facility Name", "Legal Name"]], 
        how="left", 
        left_on="name", 
        right_on="Facility Name"
    )
    .assign(
        is_brightspring = lambda df: df["Legal Name"]
        .isin([
            "VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",
            "RSCR WEST VIRGINIA, INCORPORATED"
        ])
    )
    .rename(columns={"Legal Name": "legal_owner"})
    .set_index("provider_id")
)

In [12]:
# Setting manually the data for the one closed facility
wv_facs_with_owner.at["51G023", "owner"] = "RSCR WEST VIRGINIA, INCORPORATED"
wv_facs_with_owner.at["51G023", "is_brightspring"] = True

In [13]:
wv_facs_with_owner

,name,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,Facility Name,legal_owner,is_brightspring,owner
provider_id,,,,,,,,,,,,,,,,
51G002,5TH AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"916 FIFTH AVENUE\nSAINT ALBANS, WV 25177",304 720-0015,1981-10-06,9,No,For Profit,NaN,NaN,5TH AVENUE GROUP HOME,"RSCR WEST VIRGINIA, INCORPORATED",True,NaN
51G003,POTOMAC CENTER,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"ONE BLUE STREET\nROMNEY, WV 26757",304 822-3861,1983-11-09,24,No,Non-Profit,NaN,NaN,POTOMAC CENTER,POTOMAC CENTER,False,NaN
51G007,CROSS LANES GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"5202 LINDA VISTA DRIVE\nCROSS LANES, WV 25313",304 776-2005,1986-11-17,8,No,For Profit,NaN,NaN,CROSS LANES GROUP HOME,"RSCR WEST VIRGINIA, INCORPORATED",True,NaN
51G008,EIGHTH AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"1519 8TH AVENUE\nHUNTINGTON, WV 25701",304 523-0177,1987-03-06,8,No,For Profit,NaN,NaN,EIGHTH AVENUE GROUP HOME,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True,NaN
51G009,VIRGINIA AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"821 VIRGINIA AVENUE\nHUNTINGTON, WV 25701",304 523-0196,1987-03-06,8,No,For Profit,NaN,NaN,VIRGINIA AVENUE GROUP HOME,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51G073,CHAFIN GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"6905 MERRITTS CREEK ROAD\nHUNTINGTON, WV 25702",304 733-0114,2008-10-03,8,No,For Profit,NaN,NaN,CHAFIN GROUP HOME,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True,NaN
51G074,MCGHEE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,6961 RIGHT FORK MERRITTS CREEK ROAD\nHUNTINGTO...,304 736-4647,2008-09-05,8,No,For Profit,NaN,NaN,MCGHEE GROUP HOME,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True,NaN
51G075,RITZ AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"2302 RITZ AVENUE\nWHEELING, WV 26003",304 234-3561,2011-06-07,6,No,Non-Profit,NaN,NaN,RITZ AVENUE GROUP HOME,"NORTHWOOD HEALTH SYSTEMS, INCORPORATED",False,NaN


In [14]:
wv_facs_with_owner.loc["51G023"]

name                                          GABOYA PLACE GROUP HOME
type                Intermediate Care Facilities for Individuals w...
region                                             (III) Philadelphia
state                                                              WV
address                 201 SOUTH WATER STREET\nMARTINSBURG, WV 25401
phone                                                    304 263-4601
particip_date                                              1988-05-19
certified_beds                                                      8
hospital_based                                                     No
ownership_type                                             For Profit
termination_code                                      Vol-Merg, Close
termination_date                                           2019-12-02
Facility Name                                                     NaN
legal_owner                                                       NaN
is_brightspring     

In [15]:
wv_facs_with_owner[[
    "name", "type", "region", "state", "address",
    "phone", "address", "particip_date", "certified_beds",
    "hospital_based", "ownership_type", "termination_code",
    "termination_date", "legal_owner", "is_brightspring"
]].to_csv("../../data/ownership/final/wv.csv")

---

---

---